In [1]:
%matplotlib inline
import pylab
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import cm as cm
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_validate,cross_val_score
from scipy.stats import lognorm, kurtosis, skew, probplot, kstest, ttest_ind,f_oneway
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.preprocessing import OneHotEncoder

warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# data location
localpath = ''
localpath = 'C://Users//Shaghayegh.kazemlou//Shay//Danaher//Microbiology//Data//'
filepath = 'C://Users//Shaghayegh.kazemlou//Shay//Danaher//Microbiology//Files//'

In [3]:
neg00=pd.read_csv(localpath+"All_neg.csv")
neg00.head()

C:\Users\Shaghayegh.kazemlou\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,17,293,294) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,trackingID,testdate,SampleID,ID,strains,Org_Group,Organism_ID,anti,Drug,drug_index,...,Rate_Area_BacCount_Cy9,Rate_Area_BacCount_Cy10,Rate_Area_BacCount_Cy11,Rate_Area_BacCount_Cy12,Rate_Area_BacCount_Cy13,Rate_Area_BacCount_Cy14,Dry_Growth_Pattern,Dry_Growth_Strength,drugcode,GI
0,0,20180724,10_50241,50241,A.BAUMANNII,NEG,71,AS,Ampicillin/Sulbactam,1,...,20.606316,19.373859,18.233739,17.014418,15.857302,15.278718,NaN,NaN,AS,1
1,1,20180724,10_50241,50241,A.BAUMANNII,NEG,71,AS,Ampicillin/Sulbactam,1,...,26.259380,22.930942,20.911118,19.636686,18.360967,17.271468,NaN,NaN,AS,1
2,2,20180724,10_50241,50241,A.BAUMANNII,NEG,71,AS,Ampicillin/Sulbactam,1,...,40.795997,34.359033,27.504702,23.543900,29.800889,18.576064,NaN,NaN,AS,1
3,3,20180724,10_50241,50241,A.BAUMANNII,NEG,71,AS,Ampicillin/Sulbactam,1,...,56.748749,53.377425,66.612354,51.603108,50.265342,50.535306,NaN,NaN,AS,1
4,4,20180724,10_50241,50241,A.BAUMANNII,NEG,71,AS,Ampicillin/Sulbactam,1,...,48.878813,50.803669,52.163747,100.035068,50.253040,47.175946,NaN,NaN,AS,1


In [4]:
neg = neg00.copy()
neg['bug_drug']=neg.apply(lambda row:  str(row['SampleID'])+'_'
                              +str(row['testdate'])+'_'+str(row['drug_index']), axis=1)
neg['strain'] = neg['strains']
neg['conc'] = neg['Concentration']
neg.sort_values(by =['ID','drug_index','Concentration'], axis=0, ascending=True, inplace=True)
neg.loc[:,['testdate','trackingID','bug_drug', 'ID','drug_index','Concentration','GI']][0:50].head()

,testdate,trackingID,bug_drug,ID,drug_index,Concentration,GI
59875,20200602,59875,269B_20200602_1,269,1,0.5,1
59968,20200602,59968,269M_20200602_1,269,1,0.5,1
59876,20200602,59876,269B_20200602_1,269,1,1.0,1
59969,20200602,59969,269M_20200602_1,269,1,1.0,1
59877,20200602,59877,269B_20200602_1,269,1,2.0,1


In [5]:
neg_set1=neg[neg.Growth_BacCount_Cycle_7>=15000]
neg_set2=neg[(neg.Growth_BacCount_Cycle_7<15000)&(neg.Growth_BacCount_Cycle_7>=10000)&
            (neg.Growth_BacCount_Cycle_7/neg.Growth_BacCount_Cycle_3>=8)]
neg9cycles = pd.concat([neg_set1,neg_set2])

neg10cycles=neg[~neg.trackingID.isin(neg9cycles.trackingID.unique().tolist())]
neg10cycles=neg10cycles[neg10cycles.Growth_BacCount_Cycle_8/neg10cycles.Growth_BacCount_Cycle_3>=8]

neg11cycles=neg[(~neg.trackingID.isin(neg9cycles.trackingID.unique().tolist()))&
                (~neg.trackingID.isin(neg10cycles.trackingID.unique().tolist()))]
neg=neg11cycles.copy()

In [6]:
neg['drug_type'] = 'drug_type_B'
neg.ix[neg.Drug.isin(['Ceftazidime', 'Cefepime', 'Cefotaxime', 'Imipenem', 'Meropenem']),'drug_type'] = 'drug_type_A'
len(neg)

18382

In [7]:
neg = neg[neg.anti!='ST']
len(neg)

17368

In [8]:
neg['strain'] = neg['strains']
neg['conc'] = neg['Concentration']

In [10]:
usecols=[ 'trackingID',
         'strain','SampleID','anti','conc',    
         'GI','drugcode',
         'Concentration',      
        'strains',
         'ID','drug_type',
         'bug_drug',
                  
         'AreaSUM_Cycle_2',
         'AreaSUM_Cycle_3',
         'AreaSUM_Cycle_4',
         'AreaSUM_Cycle_5',
         'AreaSUM_Cycle_6',
         'AreaSUM_Cycle_7',
         'AreaSUM_Cycle_8',
         'AreaSUM_Cycle_9',
          'AreaSUM_Cycle_10',
          'AreaSUM_Cycle_11',
#          'AreaSUM_Cycle_12',
#          'AreaSUM_Cycle_13',
#          'AreaSUM_Cycle_14',
#          'AreaSUM_Cycle_15',
         'BacCount_Cycle_2',
         'BacCount_Cycle_3',
         'BacCount_Cycle_4',
         'BacCount_Cycle_5',
         'BacCount_Cycle_6',
         'BacCount_Cycle_7',
         'BacCount_Cycle_8',
         'BacCount_Cycle_9',
          'BacCount_Cycle_10',
          'BacCount_Cycle_11',
#          'BacCount_Cycle_12',
#          'BacCount_Cycle_13',
#          'BacCount_Cycle_14',
#          'BacCount_Cycle_15',
         'Growth_AreaSUM_Cycle_2',
         'Growth_AreaSUM_Cycle_3',
         'Growth_AreaSUM_Cycle_4',
         'Growth_AreaSUM_Cycle_5',
         'Growth_AreaSUM_Cycle_6',
         'Growth_AreaSUM_Cycle_7',
         'Growth_AreaSUM_Cycle_8',
         'Growth_AreaSUM_Cycle_9',
          'Growth_AreaSUM_Cycle_10',
          'Growth_AreaSUM_Cycle_11',
#          'Growth_AreaSUM_Cycle_12',
#          'Growth_AreaSUM_Cycle_13',
#          'Growth_AreaSUM_Cycle_14',
#          'Growth_AreaSUM_Cycle_15',
         'Growth_BacCount_Cycle_2',
         'Growth_BacCount_Cycle_3',
         'Growth_BacCount_Cycle_4',
         'Growth_BacCount_Cycle_5',
         'Growth_BacCount_Cycle_6',
         'Growth_BacCount_Cycle_7',
         'Growth_BacCount_Cycle_8',
         'Growth_BacCount_Cycle_9',
          'Growth_BacCount_Cycle_10',
          'Growth_BacCount_Cycle_11',
#          'Growth_BacCount_Cycle_12',
#          'Growth_BacCount_Cycle_13',
#          'Growth_BacCount_Cycle_14',
#          'Growth_BacCount_Cycle_15',
    
    
#        'Rate_Area_Cy4vsCy3',
#        'Rate_Area_Cy3vsCy2',
#         'Rate_Area_Cy5vsCy4',
#         'Rate_Area_Cy6vsCy5',
#         'Rate_Area_Cy7vsCy6',
#         'Rate_Area_Cy8vsCy7',
#         'Rate_Area_Cy9vsCy8',
 #        'Rate_Area_Cy10vsCy9',
#          'Rate_Area_Cy11vsCy10',
#          'Rate_Area_Cy12vsCy11',
#          'Rate_Area_Cy13vsCy12',
#          'Rate_Area_Cy14vsCy13',
#          'Rate_Area_Cy15vsCy14',
#         'Rate_BacCount_Cy3vsCy2',
#         'Rate_BacCount_Cy4vsCy3',
#         'Rate_BacCount_Cy5vsCy4',
#         'Rate_BacCount_Cy6vsCy5',
#         'Rate_BacCount_Cy7vsCy6',
#         'Rate_BacCount_Cy8vsCy7',
#         'Rate_BacCount_Cy9vsCy8',
#         'Rate_BacCount_Cy10vsCy9',
#          'Rate_BacCount_Cy11vsCy10',
#          'Rate_BacCount_Cy12vsCy11',
#          'Rate_BacCount_Cy13vsCy12',
#          'Rate_BacCount_Cy14vsCy13',
#          'Rate_BacCount_Cy15vsCy14',
    
        'PerimMean_Cycle_2', 'PerimMean_Cycle_3', 'PerimMean_Cycle_4',
       'PerimMean_Cycle_5', 'PerimMean_Cycle_6', 'PerimMean_Cycle_7',
       'PerimMean_Cycle_8', 'PerimMean_Cycle_9', 'PerimMean_Cycle_10','PerimMean_Cycle_11',       
         'CircMean_Cycle_2', 'CircMean_Cycle_3', 'CircMean_Cycle_4', 
         'CircMean_Cycle_5', 'CircMean_Cycle_6', 'CircMean_Cycle_7', 
         'CircMean_Cycle_8','CircMean_Cycle_9','CircMean_Cycle_10','CircMean_Cycle_11',
         'RoundMean_Cycle_2', 'RoundMean_Cycle_3', 'RoundMean_Cycle_4', 
         'RoundMean_Cycle_5','RoundMean_Cycle_6', 'RoundMean_Cycle_7',
         'RoundMean_Cycle_8', 'RoundMean_Cycle_9','RoundMean_Cycle_10','RoundMean_Cycle_11',
         'SolidityMean_Cycle_2','SolidityMean_Cycle_3', 'SolidityMean_Cycle_4',
         'SolidityMean_Cycle_5', 'SolidityMean_Cycle_6', 'SolidityMean_Cycle_7', 
         'SolidityMean_Cycle_8', 'SolidityMean_Cycle_9', 'SolidityMean_Cycle_10', 'SolidityMean_Cycle_11',
          'LumiMean_Cycle_2', 'LumiMean_Cycle_3','LumiMean_Cycle_4', 
         'LumiMean_Cycle_5', 'LumiMean_Cycle_6','LumiMean_Cycle_7', 
         'LumiMean_Cycle_8', 'LumiMean_Cycle_9','LumiMean_Cycle_10','LumiMean_Cycle_11' ]

#choose which columns to use #pos & neg
#pos2=pos[usecols].copy()
neg2=neg[usecols].copy()
neg2.head()

,trackingID,strain,SampleID,anti,conc,GI,drugcode,Concentration,strains,ID,...,LumiMean_Cycle_2,LumiMean_Cycle_3,LumiMean_Cycle_4,LumiMean_Cycle_5,LumiMean_Cycle_6,LumiMean_Cycle_7,LumiMean_Cycle_8,LumiMean_Cycle_9,LumiMean_Cycle_10,LumiMean_Cycle_11
31061,31061,S.MARCESCENS,01_16141_A,AS,0.5,1,AS,0.5,S.MARCESCENS,16141,...,89.90,89.84,89.86,89.51,88.74,86.57,82.73,76.51,71.58,66.53
31062,31062,S.MARCESCENS,01_16141_A,AS,1.0,1,AS,1.0,S.MARCESCENS,16141,...,87.16,87.04,86.96,86.78,86.02,83.98,80.17,74.57,69.28,64.57
31063,31063,S.MARCESCENS,01_16141_A,AS,2.0,1,AS,2.0,S.MARCESCENS,16141,...,84.72,84.67,84.69,84.34,83.59,81.29,77.07,71.60,65.81,60.95
31064,31064,S.MARCESCENS,01_16141_A,AS,4.0,1,AS,4.0,S.MARCESCENS,16141,...,88.27,88.13,88.12,87.86,86.96,84.88,81.31,75.43,70.62,65.66
31065,31065,S.MARCESCENS,01_16141_A,AS,8.0,1,AS,8.0,S.MARCESCENS,16141,...,87.97,87.83,87.75,87.47,86.51,84.18,79.44,72.93,67.30,62.20


In [11]:
#cratea rate of increase for areasome and bac count
#AreaSum
keepcols6=[]
for i in range(2,11):
    colname='Rate_Area_Cy'+ str(i+1)+'vsCy'+str(i)
    keepcols6.append(colname)
    fbig='AreaSUM_Cycle_'+str(i+1)
    fsmol='AreaSUM_Cycle_'+str(i)
    neg2[colname]=(neg2[fbig]-neg2[fsmol])/neg2[fsmol]
    
#Count
keepcols7=[]
for i in range(2,11):
    colname='Rate_BacCount_Cy'+ str(i+1)+'vsCy'+str(i)
    keepcols7.append(colname)
    fbig='BacCount_Cycle_'+str(i+1)
    fsmol='BacCount_Cycle_'+str(i)
    neg2[colname]=(neg2[fbig]-neg2[fsmol])/neg2[fsmol]
    
    

#cratea rate of increase for growth columns #neg
#AreaSum
keepcols3=[]
for i in range(2,11):
    colname='Rate_Area_GCy'+ str(i+1)+'vsCy'+str(i)
    keepcols3.append(colname)
    fbig='Growth_AreaSUM_Cycle_'+str(i+1)
    fsmol='Growth_AreaSUM_Cycle_'+str(i)
    neg2[colname]=(neg2[fbig]-neg2[fsmol])/neg2[fsmol]

#Count
keepcols4=[]
for i in range(2,11):
    colname='Rate_BacCount_GCy'+ str(i+1)+'vsCy'+str(i)
    keepcols4.append(colname)
    fbig='Growth_BacCount_Cycle_'+str(i+1)
    fsmol='Growth_BacCount_Cycle_'+str(i)
    neg2[colname]=(neg2[fbig]-neg2[fsmol])/neg2[fsmol]


#cratea ratio of growth to drug for all rate columns #neg
#AreaSum
keepcols1=[]    
for i in range(2,11):
    colname='Ratio_Area_GvsD_GCy'+ str(i+1)+'vsCy'+str(i)
    keepcols1.append(colname)
    fgrowth='Rate_Area_GCy'+ str(i+1)+'vsCy'+str(i)
    fdrug='Rate_Area_Cy'+ str(i+1)+'vsCy'+str(i)
    neg2[colname]=neg2[fgrowth]/neg2[fdrug]
    
#Count   
keepcols2=[]
for i in range(2,11):
    colname='Ratio_BacCount_GvsD_Cy'+ str(i+1)+'vsCy'+str(i)
    keepcols2.append(colname)
    fgrowth='Rate_BacCount_GCy'+ str(i+1)+'vsCy'+str(i)
    fdrug='Rate_BacCount_Cy'+ str(i+1)+'vsCy'+str(i)
    neg2[colname]=neg2[fgrowth]/neg2[fdrug]
    

keepcols5=[]
for i in range(6,12):
    colname= 'Slope_Area_G' +  str(i)
    keepcols5.append(colname)
    value='Growth_AreaSUM_Cycle_'+str(i)
    previousvalue='Growth_AreaSUM_Cycle_'+str(i-1)
    neg2[colname]=(neg2[value]-neg2[previousvalue])/(neg2['Growth_AreaSUM_Cycle_3']-neg2['Growth_AreaSUM_Cycle_2'])

In [12]:
#which columns to use for modeling
generalcols=['trackingID','GI','drugcode','Concentration','ID','drug_type',
             'bug_drug','strains',
            'strain','SampleID','anti','conc']
datacolumns = ['AreaSUM_Cycle_2',
         'AreaSUM_Cycle_3',
         'AreaSUM_Cycle_4',
         'AreaSUM_Cycle_5',
         'AreaSUM_Cycle_6',
         'AreaSUM_Cycle_7',
         'AreaSUM_Cycle_8',
         'AreaSUM_Cycle_9',
        'AreaSUM_Cycle_10',
        'AreaSUM_Cycle_11',
         'BacCount_Cycle_2',
         'BacCount_Cycle_3',
         'BacCount_Cycle_4',
         'BacCount_Cycle_5',
         'BacCount_Cycle_6',
         'BacCount_Cycle_7',
         'BacCount_Cycle_8',
         'BacCount_Cycle_9',
        'BacCount_Cycle_10',
        'BacCount_Cycle_11',
         'PerimMean_Cycle_2', 'PerimMean_Cycle_3', 'PerimMean_Cycle_4',
       'PerimMean_Cycle_5', 'PerimMean_Cycle_6', 'PerimMean_Cycle_7',
       'PerimMean_Cycle_8', 'PerimMean_Cycle_9', 'PerimMean_Cycle_10','PerimMean_Cycle_11',       
         'CircMean_Cycle_2', 'CircMean_Cycle_3', 'CircMean_Cycle_4', 
         'CircMean_Cycle_5', 'CircMean_Cycle_6', 'CircMean_Cycle_7', 
         'CircMean_Cycle_8','CircMean_Cycle_9','CircMean_Cycle_10','CircMean_Cycle_11',
         'RoundMean_Cycle_2', 'RoundMean_Cycle_3', 'RoundMean_Cycle_4', 
         'RoundMean_Cycle_5','RoundMean_Cycle_6', 'RoundMean_Cycle_7',
         'RoundMean_Cycle_8', 'RoundMean_Cycle_9','RoundMean_Cycle_10','RoundMean_Cycle_11',
         'SolidityMean_Cycle_2','SolidityMean_Cycle_3', 'SolidityMean_Cycle_4',
         'SolidityMean_Cycle_5', 'SolidityMean_Cycle_6', 'SolidityMean_Cycle_7', 
         'SolidityMean_Cycle_8', 'SolidityMean_Cycle_9', 'SolidityMean_Cycle_10', 'SolidityMean_Cycle_11',
          'LumiMean_Cycle_2', 'LumiMean_Cycle_3','LumiMean_Cycle_4', 
         'LumiMean_Cycle_5', 'LumiMean_Cycle_6','LumiMean_Cycle_7', 
         'LumiMean_Cycle_8', 'LumiMean_Cycle_9','LumiMean_Cycle_10','LumiMean_Cycle_11' ]

main_cols=generalcols+datacolumns+keepcols1+keepcols2+keepcols3+keepcols4+keepcols5+keepcols6+keepcols7

#separate dataset for modeling #pos & neg
neg3=neg2[main_cols].copy()

model_data_neg=neg3.copy()

model_data_neg.head()

,trackingID,GI,drugcode,Concentration,ID,drug_type,bug_drug,strains,strain,SampleID,...,Rate_Area_Cy11vsCy10,Rate_BacCount_Cy3vsCy2,Rate_BacCount_Cy4vsCy3,Rate_BacCount_Cy5vsCy4,Rate_BacCount_Cy6vsCy5,Rate_BacCount_Cy7vsCy6,Rate_BacCount_Cy8vsCy7,Rate_BacCount_Cy9vsCy8,Rate_BacCount_Cy10vsCy9,Rate_BacCount_Cy11vsCy10
31061,31061,1,AS,0.5,16141,drug_type_B,01_16141_A_20181029_1,S.MARCESCENS,S.MARCESCENS,01_16141_A,...,0.091340,-0.624842,-0.476351,0.448387,1.014477,1.401327,1.143877,0.919682,0.673789,0.285724
31062,31062,1,AS,1.0,16141,drug_type_B,01_16141_A_20181029_1,S.MARCESCENS,S.MARCESCENS,01_16141_A,...,0.094788,-0.872630,-0.456415,0.351351,1.097333,1.554355,1.213539,0.955363,0.650336,0.307690
31063,31063,1,AS,2.0,16141,drug_type_B,01_16141_A_20181029_1,S.MARCESCENS,S.MARCESCENS,01_16141_A,...,0.065753,0.783019,0.894180,1.053073,1.136054,1.876433,1.176484,0.982399,0.632076,0.262665
31064,31064,1,AS,4.0,16141,drug_type_B,01_16141_A_20181029_1,S.MARCESCENS,S.MARCESCENS,01_16141_A,...,0.110460,-0.876597,-0.405797,0.391986,1.137672,1.368267,1.167862,0.939902,0.708424,0.320556
31065,31065,1,AS,8.0,16141,drug_type_B,01_16141_A_20181029_1,S.MARCESCENS,S.MARCESCENS,01_16141_A,...,0.046554,0.253165,0.858586,1.192935,1.254027,1.506322,1.124589,0.963865,0.630165,0.270373


In [13]:

#neg
model_data_neg = model_data_neg.replace([np.inf,-np.inf], np.nan)
model_data_neg.dropna(inplace=True)



In [14]:
# run the model for gram pos data
onemodel_n=[]
onemodel_f1_results_train=[]
onemodel_f1_results_test=[]
onemodel_auc_results=[]
roc_df_big=pd.DataFrame()

n=20
dfgram=neg2
originaldf=neg
clf = RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators=600, min_samples_leaf=n)
clf = RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators=600, min_samples_leaf=n,
                            class_weight='balanced')

clf = RandomForestClassifier(n_jobs=-1, random_state=0, max_features=16, n_estimators=90,
                             min_samples_leaf=n,
                            class_weight='balanced')

# model with independant train test split with each drug one hot encoded
model_data4=model_data_neg

model_data4 = model_data4.replace([np.inf,-np.inf], np.nan)
model_data4.dropna(inplace=True)
# generate binary values using get_dummies
df1=model_data4.copy()

#df = pd.get_dummies(df1, columns=["bug_type","drug_type"], prefix=[None, None] )
df = pd.get_dummies(df1, columns=["strains","drug_type"], prefix=[None, None] )

In [15]:
#choose feature set and target 
#pos & neg
features = ['Concentration', 
        'Ratio_Area_GvsD_GCy3vsCy2',
        'Ratio_Area_GvsD_GCy4vsCy3', 'Ratio_Area_GvsD_GCy5vsCy4',
        'Ratio_Area_GvsD_GCy6vsCy5', 'Ratio_Area_GvsD_GCy7vsCy6',
        'Ratio_Area_GvsD_GCy8vsCy7', 'Ratio_Area_GvsD_GCy9vsCy8','Ratio_Area_GvsD_GCy10vsCy9','Ratio_Area_GvsD_GCy11vsCy10',
            
        'Ratio_BacCount_GvsD_Cy3vsCy2', 'Ratio_BacCount_GvsD_Cy4vsCy3',
        'Ratio_BacCount_GvsD_Cy5vsCy4', 'Ratio_BacCount_GvsD_Cy6vsCy5',
        'Ratio_BacCount_GvsD_Cy7vsCy6', 'Ratio_BacCount_GvsD_Cy8vsCy7',
        'Ratio_BacCount_GvsD_Cy9vsCy8','Ratio_BacCount_GvsD_Cy10vsCy9', 'Ratio_BacCount_GvsD_Cy11vsCy10', 
            
        'Rate_Area_GCy3vsCy2',
        'Rate_Area_GCy4vsCy3', 'Rate_Area_GCy5vsCy4', 'Rate_Area_GCy6vsCy5',
        'Rate_Area_GCy7vsCy6', 'Rate_Area_GCy8vsCy7', 'Rate_Area_GCy9vsCy8','Rate_Area_GCy10vsCy9','Rate_Area_GCy11vsCy10',

        'Rate_BacCount_GCy3vsCy2', 'Rate_BacCount_GCy4vsCy3',
        'Rate_BacCount_GCy5vsCy4', 'Rate_BacCount_GCy6vsCy5',
        'Rate_BacCount_GCy7vsCy6', 'Rate_BacCount_GCy8vsCy7',
        'Rate_BacCount_GCy9vsCy8', 'Rate_BacCount_GCy10vsCy9', 'Rate_BacCount_GCy11vsCy10', 
            
            
        'Rate_Area_Cy3vsCy2',
        'Rate_Area_Cy4vsCy3',
        'Rate_Area_Cy5vsCy4',
         'Rate_Area_Cy6vsCy5',
         'Rate_Area_Cy7vsCy6',
         'Rate_Area_Cy8vsCy7',
         'Rate_Area_Cy9vsCy8','Rate_Area_Cy10vsCy9','Rate_Area_Cy11vsCy10',

         'Rate_BacCount_Cy3vsCy2',
         'Rate_BacCount_Cy4vsCy3',
         'Rate_BacCount_Cy5vsCy4',
         'Rate_BacCount_Cy6vsCy5',
         'Rate_BacCount_Cy7vsCy6',
         'Rate_BacCount_Cy8vsCy7',
         'Rate_BacCount_Cy9vsCy8','Rate_BacCount_Cy10vsCy9','Rate_BacCount_Cy11vsCy10',
  
            
        'Slope_Area_G6','Slope_Area_G7', 'Slope_Area_G8', 'Slope_Area_G9','Slope_Area_G10','Slope_Area_G11',
           
         'drug_type_A', 'drug_type_B',
           
         'AreaSUM_Cycle_2',
         'AreaSUM_Cycle_3',
         'AreaSUM_Cycle_4',
         'AreaSUM_Cycle_5',
         'AreaSUM_Cycle_6',
         'AreaSUM_Cycle_7',
         'AreaSUM_Cycle_8',
         'AreaSUM_Cycle_9',
            'AreaSUM_Cycle_10',
            'AreaSUM_Cycle_11',
         'BacCount_Cycle_2',
         'BacCount_Cycle_3',
         'BacCount_Cycle_4',
         'BacCount_Cycle_5',
         'BacCount_Cycle_6',
         'BacCount_Cycle_7',
         'BacCount_Cycle_8',
         'BacCount_Cycle_9',
            'BacCount_Cycle_10',
            'BacCount_Cycle_11',
        'PerimMean_Cycle_2', 'PerimMean_Cycle_3', 'PerimMean_Cycle_4',
       'PerimMean_Cycle_5', 'PerimMean_Cycle_6', 'PerimMean_Cycle_7',
       'PerimMean_Cycle_8', 'PerimMean_Cycle_9', 'PerimMean_Cycle_10','PerimMean_Cycle_11',       
         'CircMean_Cycle_2', 'CircMean_Cycle_3', 'CircMean_Cycle_4', 
         'CircMean_Cycle_5', 'CircMean_Cycle_6', 'CircMean_Cycle_7', 
         'CircMean_Cycle_8','CircMean_Cycle_9','CircMean_Cycle_10','CircMean_Cycle_11',
         'RoundMean_Cycle_2', 'RoundMean_Cycle_3', 'RoundMean_Cycle_4', 
         'RoundMean_Cycle_5','RoundMean_Cycle_6', 'RoundMean_Cycle_7',
         'RoundMean_Cycle_8', 'RoundMean_Cycle_9','RoundMean_Cycle_10','RoundMean_Cycle_11',
         'SolidityMean_Cycle_2','SolidityMean_Cycle_3', 'SolidityMean_Cycle_4',
         'SolidityMean_Cycle_5', 'SolidityMean_Cycle_6', 'SolidityMean_Cycle_7', 
         'SolidityMean_Cycle_8', 'SolidityMean_Cycle_9', 'SolidityMean_Cycle_10', 'SolidityMean_Cycle_11',
          'LumiMean_Cycle_2', 'LumiMean_Cycle_3','LumiMean_Cycle_4', 
         'LumiMean_Cycle_5', 'LumiMean_Cycle_6','LumiMean_Cycle_7', 
         'LumiMean_Cycle_8', 'LumiMean_Cycle_9','LumiMean_Cycle_10','LumiMean_Cycle_11']
       
target= 'GI'
#target= 'MIC'

len(features)

133

In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

normalfeatures = ['Concentration', 
        'Ratio_Area_GvsD_GCy3vsCy2',
        'Ratio_Area_GvsD_GCy4vsCy3', 'Ratio_Area_GvsD_GCy5vsCy4',
        'Ratio_Area_GvsD_GCy6vsCy5', 'Ratio_Area_GvsD_GCy7vsCy6',
        'Ratio_Area_GvsD_GCy8vsCy7', 'Ratio_Area_GvsD_GCy9vsCy8','Ratio_Area_GvsD_GCy10vsCy9','Ratio_Area_GvsD_GCy11vsCy10',
            
        'Ratio_BacCount_GvsD_Cy3vsCy2', 'Ratio_BacCount_GvsD_Cy4vsCy3',
        'Ratio_BacCount_GvsD_Cy5vsCy4', 'Ratio_BacCount_GvsD_Cy6vsCy5',
        'Ratio_BacCount_GvsD_Cy7vsCy6', 'Ratio_BacCount_GvsD_Cy8vsCy7',
        'Ratio_BacCount_GvsD_Cy9vsCy8','Ratio_BacCount_GvsD_Cy10vsCy9', 'Ratio_BacCount_GvsD_Cy11vsCy10', 
            
        'Rate_Area_GCy3vsCy2',
        'Rate_Area_GCy4vsCy3', 'Rate_Area_GCy5vsCy4', 'Rate_Area_GCy6vsCy5',
        'Rate_Area_GCy7vsCy6', 'Rate_Area_GCy8vsCy7', 'Rate_Area_GCy9vsCy8','Rate_Area_GCy10vsCy9','Rate_Area_GCy11vsCy10',

        'Rate_BacCount_GCy3vsCy2', 'Rate_BacCount_GCy4vsCy3',
        'Rate_BacCount_GCy5vsCy4', 'Rate_BacCount_GCy6vsCy5',
        'Rate_BacCount_GCy7vsCy6', 'Rate_BacCount_GCy8vsCy7',
        'Rate_BacCount_GCy9vsCy8', 'Rate_BacCount_GCy10vsCy9', 'Rate_BacCount_GCy11vsCy10', 
            
            
        'Rate_Area_Cy3vsCy2',
        'Rate_Area_Cy4vsCy3',
        'Rate_Area_Cy5vsCy4',
         'Rate_Area_Cy6vsCy5',
         'Rate_Area_Cy7vsCy6',
         'Rate_Area_Cy8vsCy7',
         'Rate_Area_Cy9vsCy8','Rate_Area_Cy10vsCy9','Rate_Area_Cy11vsCy10',

         'Rate_BacCount_Cy3vsCy2',
         'Rate_BacCount_Cy4vsCy3',
         'Rate_BacCount_Cy5vsCy4',
         'Rate_BacCount_Cy6vsCy5',
         'Rate_BacCount_Cy7vsCy6',
         'Rate_BacCount_Cy8vsCy7',
         'Rate_BacCount_Cy9vsCy8','Rate_BacCount_Cy10vsCy9','Rate_BacCount_Cy11vsCy10',
  
            
        'Slope_Area_G6','Slope_Area_G7', 'Slope_Area_G8', 'Slope_Area_G9','Slope_Area_G10','Slope_Area_G11',
           
           
         'AreaSUM_Cycle_2',
         'AreaSUM_Cycle_3',
         'AreaSUM_Cycle_4',
         'AreaSUM_Cycle_5',
         'AreaSUM_Cycle_6',
         'AreaSUM_Cycle_7',
         'AreaSUM_Cycle_8',
         'AreaSUM_Cycle_9',
            'AreaSUM_Cycle_10',
            'AreaSUM_Cycle_11',
         'BacCount_Cycle_2',
         'BacCount_Cycle_3',
         'BacCount_Cycle_4',
         'BacCount_Cycle_5',
         'BacCount_Cycle_6',
         'BacCount_Cycle_7',
         'BacCount_Cycle_8',
         'BacCount_Cycle_9',
            'BacCount_Cycle_10',
            'BacCount_Cycle_11',
        'PerimMean_Cycle_2', 'PerimMean_Cycle_3', 'PerimMean_Cycle_4',
       'PerimMean_Cycle_5', 'PerimMean_Cycle_6', 'PerimMean_Cycle_7',
       'PerimMean_Cycle_8', 'PerimMean_Cycle_9', 'PerimMean_Cycle_10','PerimMean_Cycle_11',       
         'CircMean_Cycle_2', 'CircMean_Cycle_3', 'CircMean_Cycle_4', 
         'CircMean_Cycle_5', 'CircMean_Cycle_6', 'CircMean_Cycle_7', 
         'CircMean_Cycle_8','CircMean_Cycle_9','CircMean_Cycle_10','CircMean_Cycle_11',
         'RoundMean_Cycle_2', 'RoundMean_Cycle_3', 'RoundMean_Cycle_4', 
         'RoundMean_Cycle_5','RoundMean_Cycle_6', 'RoundMean_Cycle_7',
         'RoundMean_Cycle_8', 'RoundMean_Cycle_9','RoundMean_Cycle_10','RoundMean_Cycle_11',
         'SolidityMean_Cycle_2','SolidityMean_Cycle_3', 'SolidityMean_Cycle_4',
         'SolidityMean_Cycle_5', 'SolidityMean_Cycle_6', 'SolidityMean_Cycle_7', 
         'SolidityMean_Cycle_8', 'SolidityMean_Cycle_9', 'SolidityMean_Cycle_10', 'SolidityMean_Cycle_11',
          'LumiMean_Cycle_2', 'LumiMean_Cycle_3','LumiMean_Cycle_4', 
         'LumiMean_Cycle_5', 'LumiMean_Cycle_6','LumiMean_Cycle_7', 
         'LumiMean_Cycle_8', 'LumiMean_Cycle_9','LumiMean_Cycle_10','LumiMean_Cycle_11'
]



scaler = StandardScaler()

df_notnormalized = df.copy()
df[normalfeatures] =scaler.fit_transform(df[normalfeatures]) 

In [17]:

#model on all antibiotics with drug one hot encoded

# read test set
testset=pd.read_csv(filepath+'testset-neg-V0.csv',names=['bug_drug'],header=None)
testset.reset_index(drop=True, inplace=True)
testlist=testset.bug_drug.tolist()

train_df = df[~df.bug_drug.isin(testlist)]
test_df = df[df.bug_drug.isin(testlist)]

train_df = train_df.copy().reset_index().rename(columns={'index':'later_index'})
test_df = test_df.copy().reset_index().rename(columns={'index':'later_index'})



X_train = np.array(train_df[features])
y_train = np.array(train_df[target])

X_test = np.array(test_df[features])
y_test = np.array(test_df[target])


# Create a random forest Classifier. By convention, clf means 'Classifier'
clf.fit(X_train, y_train)
#Predict the target variable for the training & testing dataset using the best algorithm


train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

#f1 score
trainf1=metrics.f1_score(y_train,train_predictions)
testf1=metrics.f1_score(y_test,test_predictions)
onemodel_n.append(n)

onemodel_f1_results_train.append(trainf1)
onemodel_f1_results_test.append(testf1)

fpr, tpr, thresholds = metrics.roc_curve(y_test,test_predictions)
auc=metrics.auc(fpr, tpr)
onemodel_auc_results.append(auc)

#roc curve
test_predicted_prob = clf.predict_proba(X_test)
scores = test_predicted_prob[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(y_test, scores)
roc_df = pd.DataFrame({'min_samples_leaf':n,'FPR' : fpr, 'TPR' : tpr, 'Threshold' : thresholds})


#train and test dataset with predictions attached
train_df_pred=pd.concat([train_df,pd.DataFrame(data={'GI_pred': train_predictions})],axis=1)
test_df_pred=pd.concat([test_df,pd.DataFrame(data={'GI_pred': test_predictions})],axis=1)

train_df_pred['data_type']='train'
test_df_pred['data_type']='test'

final_df_temp=pd.concat([train_df_pred,test_df_pred])

final_df=final_df_temp.copy()
final_df=final_df[~final_df.data_type.isnull()]
final_df=final_df[~final_df.GI_pred.isnull()]

len(X_train)+len(X_test),len(X_train),len(X_test)

(16829, 12569, 4260)

In [18]:
onemodel_results=pd.DataFrame({'mean_leaf_node':onemodel_n,
                               'train_f1':onemodel_f1_results_train,
                               'test_f1':onemodel_f1_results_test,
                               'auc':onemodel_auc_results,
                                  })

#output model performance results
# onemodel_results.to_csv('onemodel_results_neg.csv')

#output the roc curve data
#roc_df.to_csv('onemodel_roc_neg.csv')

#output final df including predictions
#final_df.to_csv('final_df_neg.csv',index=False)
onemodel_results

,mean_leaf_node,train_f1,test_f1,auc
0,20,0.925176,0.877588,0.887349


In [19]:
names= features


importance=clf.feature_importances_
print (sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names), reverse=True))
ee=sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names), reverse=True)
importance
ee = pd.DataFrame(ee)
ee.columns={'contribution','variable'}
ee[0:30]

[(0.0798, 'BacCount_Cycle_11'), (0.0704, 'Concentration'), (0.0671, 'AreaSUM_Cycle_11'), (0.0568, 'Rate_Area_Cy10vsCy9'), (0.0548, 'LumiMean_Cycle_11'), (0.0539, 'Rate_Area_Cy9vsCy8'), (0.0474, 'Rate_Area_Cy11vsCy10'), (0.0296, 'AreaSUM_Cycle_10'), (0.0267, 'LumiMean_Cycle_10'), (0.025, 'Ratio_Area_GvsD_GCy8vsCy7'), (0.0234, 'Rate_BacCount_Cy10vsCy9'), (0.0233, 'Rate_Area_Cy8vsCy7'), (0.0214, 'Ratio_Area_GvsD_GCy9vsCy8'), (0.0213, 'BacCount_Cycle_10'), (0.0208, 'BacCount_Cycle_9'), (0.0207, 'Rate_BacCount_Cy11vsCy10'), (0.0189, 'Rate_BacCount_Cy9vsCy8'), (0.0116, 'AreaSUM_Cycle_9'), (0.009, 'RoundMean_Cycle_7'), (0.009, 'RoundMean_Cycle_10'), (0.0089, 'RoundMean_Cycle_8'), (0.0087, 'RoundMean_Cycle_11'), (0.0082, 'SolidityMean_Cycle_9'), (0.008, 'Rate_Area_GCy8vsCy7'), (0.0079, 'RoundMean_Cycle_6'), (0.0072, 'CircMean_Cycle_7'), (0.0071, 'SolidityMean_Cycle_8'), (0.0068, 'SolidityMean_Cycle_7'), (0.0066, 'Ratio_Area_GvsD_GCy7vsCy6'), (0.0064, 'RoundMean_Cycle_9'), (0.0062, 'Rate_BacCou

,variable,contribution
0,0.0798,BacCount_Cycle_11
1,0.0704,Concentration
2,0.0671,AreaSUM_Cycle_11
3,0.0568,Rate_Area_Cy10vsCy9
4,0.0548,LumiMean_Cycle_11
5,0.0539,Rate_Area_Cy9vsCy8
6,0.0474,Rate_Area_Cy11vsCy10
7,0.0296,AreaSUM_Cycle_10
8,0.0267,LumiMean_Cycle_10
9,0.0250,Ratio_Area_GvsD_GCy8vsCy7


# prepare output

In [20]:

predict_testset=pd.merge(test_df_pred.loc[:,['trackingID','GI_pred']], df_notnormalized, on = 'trackingID', how='left')
predict_testset['false_positive']=np.where((predict_testset['GI']==0)&(predict_testset['GI_pred']==1),1,0)
predict_testset['false_negative']=np.where((predict_testset['GI']==1)&(predict_testset['GI_pred']==0),1,0)

# EA 

In [21]:
growthThresh = 0.5

In [22]:
#!pip install opencv-python

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import csv
import xlsxwriter
from tabulate import tabulate
import pandas as pd
from pprint import pprint

import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

from AST_Infer_Utils import *

In [24]:
evaluation_df = pd.concat([test_df, train_df])
evaluation_df=test_df.copy()

In [ ]:
resetInferDataDict()
yTruthAll = np.array([])
yPredAll = np.array([])
infoListAll=np.array([])


for index, row in evaluation_df.iterrows():
    numFeatures = row[features]
    yTruth = row['GI']
    infoList = ['', 0, row['strain'], row['ID'], row['anti'], row['conc'], '0']
    
#    print(row['strain'])
    
    
    yPred = clf.predict([numFeatures])
    
    infoList = [infoList]
    
    yTruth = np.array(yTruth).reshape(1,1)
    yPred = np.array(yPred).reshape(1,1)

    
    
    if yTruthAll.size!=0:
        yTruthAll=np.concatenate([yTruthAll, yTruth], axis=0)
    else:
        yTruthAll=yTruth
    if yPredAll.size!=0:
        yPredAll=np.concatenate([yPredAll, yPred], axis=0)
    else:
        yPredAll=yPred
        
    if infoListAll.size!=0:
        infoListAll=np.concatenate([infoListAll, infoList], axis=0)
    else:
        infoListAll=np.array(infoList)

    addInferData(infoList, yTruth, yPred)
    
    
    
# not needed, this will be done in a later section
#groupCount, accAACount, accEACount, undetCount, strainSet, drugSet = compileAllInferData(growthThresh)


In [43]:
csvFileName = 'GramNeg_FullSample.csv'
# csvFileName = 'GramPos/GramPos_FullSample.csv'

dataDict = []
indMap = {}


with open(csvFileName, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)
    
    for n, line in enumerate(csvReader):            
        dataDict.append(line)
        indMap[int(line['Index'])] = n
        
dataDict[0]['data_type'] = ''
dataDict[0]['GI_pred'] = ''
        
print('totally %d valid examples' %len(dataDict))

totally 55613 valid examples


In [44]:
dataPath = 'C:/Users/Shaghayegh.kazemlou/Shay/Danaher/Microbiology/Files/Output/EA/'

predXLSXFileName = dataPath + 'RF-Model36-EA-testset-neg.xlsx'
workBook = xlsxwriter.Workbook(predXLSXFileName, {'strings_to_numbers': True, 'nan_inf_to_errors': True})

workSheet = workBook.add_worksheet('sample prediction')

workSheet.write_row(0, 0, dataDict[0].keys())
for n, row in enumerate(dataDict):
    workSheet.write_row(n+1, 0, row.values())

In [45]:
groupCount, accAACount, accEACount, undetCount, strainSet, drugSet = compileAllInferData(growthThresh)

groupCount = np.pad(groupCount, ((0,1),(0,1)), 'constant')
accAACount = np.pad(accAACount, ((0,1),(0,1)), 'constant')
accEACount = np.pad(accEACount, ((0,1),(0,1)), 'constant')
undetCount = np.pad(undetCount, ((0,1),(0,1)), 'constant')

In [46]:
strainSet

['E.COLI',
 'K.OXYTOCA',
 'S.MARCESCENS',
 'P.VULGARIS',
 'A.BAUMANNII',
 'E.AEROGENES',
 'C.FREUNDII',
 'K.PNEUMONIAE',
 'E.CLOACAE',
 'P.RETTGERI',
 'P.MIRABILIS',
 'P.AERUGINOSA']

In [47]:

yPredBinaryAll = (yPredAll > growthThresh)

sampCount = np.zeros((25, 25))
errCount = np.zeros((25, 25))

bias0Cent = 10
sampCountByBias = np.zeros(bias0Cent*2 + 1)
errCountByBias = np.zeros(bias0Cent*2 + 1)

for (yt, yp, info) in zip(yTruthAll, yPredBinaryAll, infoListAll):
    
    if info[4] not in drugSet:
        drugSet.append(info[4])

    colInd = drugSet.index(info[4])

    if info[2] not in strainSet:
        strainSet.append(info[2])

    rowInd = strainSet.index(info[2])
    
#   wellBias = int(info[6])
#   biasInd = wellBias + bias0Cent

    sampCount[rowInd, colInd] += 1
#    sampCountByBias[biasInd] += 1

    if yp != yt:   
        errCount[rowInd, colInd] += 1    
#        errCountByBias[biasInd] += 1
    
sampCount = sampCount[0:len(strainSet)+1, 0:len(drugSet)+1]
errCount = errCount[0:len(strainSet)+1, 0:len(drugSet)+1]

#biasEff = np.where(sampCountByBias > 0)
#biasLow = np.min(biasEff)
#biasHigh = np.max(biasEff)+1
#biasRange = np.arange(biasLow-bias0Cent, biasHigh-bias0Cent)
#bias0Cent -= biasLow
#sampCountByBias = sampCountByBias[biasLow:biasHigh]
#errCountByBias = errCountByBias[biasLow:biasHigh]

In [48]:
strainSet

['E.COLI',
 'K.OXYTOCA',
 'S.MARCESCENS',
 'P.VULGARIS',
 'A.BAUMANNII',
 'E.AEROGENES',
 'C.FREUNDII',
 'K.PNEUMONIAE',
 'E.CLOACAE',
 'P.RETTGERI',
 'P.MIRABILIS',
 'P.AERUGINOSA']

In [49]:
sampCount[:,-1] = np.sum(sampCount, axis=1)
sampCount[-1,:] = np.sum(sampCount, axis=0)
errCount[:,-1] = np.sum(errCount, axis=1)
errCount[-1,:] = np.sum(errCount, axis=0)
groupCount[:,-1] = np.sum(groupCount, axis=1)
groupCount[-1,:] = np.sum(groupCount, axis=0)
accAACount[:,-1] = np.sum(accAACount, axis=1)
accAACount[-1,:] = np.sum(accAACount, axis=0)
accEACount[:,-1] = np.sum(accEACount, axis=1)
accEACount[-1,:] = np.sum(accEACount, axis=0)
undetCount[:,-1] = np.sum(undetCount, axis=1)
undetCount[-1,:] = np.sum(undetCount, axis=0)

In [50]:
tableHeader = list([''] + drugSet)
tableDataCount = []
tableDataGroupCount = []
tableDataAcc = []
tableDataAccAA = []
tableDataAccEA = []
tableDataUndet = []

sampCount = sampCount.astype(int)
groupCount = groupCount.astype(int)
accRate = np.round((1 - errCount / sampCount) * 100, decimals=1)
accAARate = np.round((accAACount / groupCount) * 100, decimals=1)
accEARate = np.round((accEACount / groupCount) * 100, decimals=1)

for n, strain in enumerate(strainSet):
    tableDataCount.append([strainSet[n][0:6]] + list(sampCount[n]))
    tableDataGroupCount.append([strainSet[n][0:6]] + list(groupCount[n]))
    tableDataAcc.append([strainSet[n][0:6]] + list(accRate[n]))
    tableDataAccAA.append([strainSet[n][0:6]] + list(accAARate[n]))
    tableDataAccEA.append([strainSet[n][0:6]] + list(accEARate[n]))
    tableDataUndet.append([strainSet[n][0:6]] + list(undetCount[n]))

tableDataCount.append(['SUM'] + list(sampCount[-1,:]))
tableDataGroupCount.append(['SUM'] + list(groupCount[-1,:]))
tableDataAcc.append(['MEAN'] + list(accRate[-1,:]))
tableDataAccAA.append(['MEAN'] + list(accAARate[-1,:]))
tableDataAccEA.append(['MEAN'] + list(accEARate[-1,:]))
tableDataUndet.append(['SUM'] + list(undetCount[-1,:]))

sampCountTable = tabulate(tableDataCount, headers=tableHeader+['SUM'], tablefmt="pretty")
groupCountTable = tabulate(tableDataGroupCount, headers=tableHeader+['SUM'], tablefmt="pretty")
accRateTable = tabulate(tableDataAcc, headers=tableHeader+['MEAN'], tablefmt="pretty", floatfmt=".1f")
accAARateTable = tabulate(tableDataAccAA, headers=tableHeader+['MEAN'], tablefmt="pretty", floatfmt=".1f")
accEARateTable = tabulate(tableDataAccEA, headers=tableHeader+['MEAN'], tablefmt="pretty", floatfmt=".1f")
undetCountTable = tabulate(tableDataUndet, headers=tableHeader+['SUM'], tablefmt="pretty")

print('example count by species & drug groups')
print(sampCountTable)
print()
print('dilution group count by species & drug groups')
print(groupCountTable)
print()
print('G/I accuracy in % by species & drug groups')
print(accRateTable)
print()
print('AA accuracy in % by species & drug groups')
print(accAARateTable)
print()
print('EA accuracy in % by species & drug groups')
print(accEARateTable)
print()
print('undeter in % by species & drug groups')
print(undetCountTable)

# plt.figure(figsize=(12,8))
# sn.heatmap(accRate, annot=True, square=False)
# plt.title('accuracy in % by species & drug groups')

example count by species & drug groups
+--------+-----+-----+------+-----+------+-----+------+----+-----+-----+------+-----+------+
|        | GM  | TOB | CPFX | AS  | LVFX | IPM | MEPM | ST | CAZ | CTX | CFPM | TP  | SUM  |
+--------+-----+-----+------+-----+------+-----+------+----+-----+-----+------+-----+------+
| E.COLI | 16  | 40  |  80  | 42  |  63  | 39  |  39  | 0  | 55  | 32  |  64  | 71  | 541  |
| K.OXYT | 16  |  8  |  0   |  0  |  0   |  0  |  8   | 0  |  7  |  0  |  0   | 16  |  55  |
| S.MARC | 16  |  8  |  16  | 21  |  8   |  8  |  16  | 0  |  8  | 16  |  16  | 16  | 149  |
| P.VULG |  0  |  0  |  0   |  0  |  8   |  8  |  8   | 0  |  0  |  0  |  0   |  0  |  24  |
| A.BAUM | 24  | 64  |  24  |  7  |  32  | 56  |  39  | 0  |  8  | 15  |  23  | 39  | 331  |
| E.AERO |  0  |  0  |  0   |  0  |  0   |  0  |  0   | 0  |  0  |  0  |  8   |  0  |  8   |
| C.FREU | 32  | 31  |  15  |  0  |  32  |  8  |  8   | 0  |  8  | 15  |  40  | 24  | 213  |
| K.PNEU |  7  |  7  |  14  |  

C:\Users\Shaghayegh.kazemlou\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Shaghayegh.kazemlou\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
C:\Users\Shaghayegh.kazemlou\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [51]:
workSheet = workBook.add_worksheet('sample count by group')

workSheet.write_row(0, 0, tableHeader+['SUM'])
for n, row in enumerate(tableDataCount):
    workSheet.write_row(n+1, 0, row)

workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workSheet = workBook.add_worksheet('dilution group count by group')

workSheet.write_row(0, 0, tableHeader+['SUM'])
for n, row in enumerate(tableDataGroupCount):
    workSheet.write_row(n+1, 0, row)

workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workSheet = workBook.add_worksheet('accuracy by group')

workSheet.write_row(0, 0, tableHeader+['MEAN'])
for n, row in enumerate(tableDataAcc):
    workSheet.write_row(n+1, 0, row)
    
workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workSheet = workBook.add_worksheet('AA accuracy by group')

workSheet.write_row(0, 0, tableHeader+['MEAN'])
for n, row in enumerate(tableDataAccAA):
    workSheet.write_row(n+1, 0, row)
    
workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workSheet = workBook.add_worksheet('EA accuracy by group')

workSheet.write_row(0, 0, tableHeader+['MEAN'])
for n, row in enumerate(tableDataAccEA):
    workSheet.write_row(n+1, 0, row)
    
workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workSheet = workBook.add_worksheet('undertermined count by group')

workSheet.write_row(0, 0, tableHeader+['SUM'])
for n, row in enumerate(tableDataUndet):
    workSheet.write_row(n+1, 0, row)

workSheet.write_row(n+4, 0, ['drugs:']+drugSet)
workSheet.write_row(n+5, 0, ['species:']+strainSet)

workBook.close()